# Transportation Problem using PuLP

## Problem Statement
A company has 3 sources and 3 destinations.  
Each source has a fixed supply and each destination has a fixed demand.

The objective is to **minimize total transportation cost** while satisfying
all supply and demand constraints.


## Mathematical Model

### Decision Variables
$( x_{ij} $): units transported from source $( i $) to destination $( j $)

### Objective Function
$$
\min Z = \sum_i \sum_j c_{ij} x_{ij}
$$

### Constraints
- Supply constraints:
$$
\sum_j x_{ij} = \text{supply}_i
$$

- Demand constraints:
$$
\sum_i x_{ij} = \text{demand}_j
$$

- Non-negativity:
$$
x_{ij} \ge 0
$$


In [3]:
from pulp import *

sources = ['S1', 'S2', 'S3']
destinations = ['D1', 'D2', 'D3']

cost = {
    'S1': {'D1': 2, 'D2': 3, 'D3': 1},
    'S2': {'D1': 5, 'D2': 4, 'D3': 8},
    'S3': {'D1': 5, 'D2': 6, 'D3': 8}
}

supply = {'S1': 20, 'S2': 30, 'S3': 25}
demand = {'D1': 10, 'D2': 35, 'D3': 30}

model = LpProblem("Transportation_Problem", LpMinimize)

x = LpVariable.dicts(
    "Ship",
    (sources, destinations),
    lowBound=0,
    cat='Continuous'
)

model += lpSum(
    cost[i][j] * x[i][j]
    for i in sources
    for j in destinations
), "Total_Transportation_Cost"

for i in sources:
    model += lpSum(x[i][j] for j in destinations) == supply[i], f"Supply_{i}"

for j in destinations:
    model += lpSum(x[i][j] for i in sources) == demand[j], f"Demand_{j}"

model.solve(PULP_CBC_CMD(msg=False))

print("Status:", LpStatus[model.status])
print("Minimum Transportation Cost:", value(model.objective))

for i in sources:
    for j in destinations:
        if x[i][j].varValue > 0:
            print(f"Ship {x[i][j].varValue} units from {i} to {j}")

Status: Optimal
Minimum Transportation Cost: 300.0
Ship 20.0 units from S1 to D3
Ship 30.0 units from S2 to D2
Ship 10.0 units from S3 to D1
Ship 5.0 units from S3 to D2
Ship 10.0 units from S3 to D3


## Interpretation of Results

- The solver status is **Optimal**
- The objective value represents the **minimum total transportation cost**
- Only non-zero shipments are displayed
- All supply and demand constraints are satisfied

This confirms the correctness of the LP formulation.
